In [3]:
import os
import django
import pandas as pd
import http
import json
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'fxfutbol.settings')
django.setup()
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [60]:
from mymodel.models import Tournament 
from mymodel.models import Season
from mymodel.models import Team
# Modelleri import edin
import pandas as pd

class SofascoreAPI:
    def __init__(self, date):
        self.date = date
        
    def fetch_data(self):
        """Verilen tarih için sofascore API'den veri çekme işlemi."""
        conn = http.client.HTTPSConnection('www.sofascore.com')
        conn.request(
            'GET',
            f'/api/v1/sport/football/scheduled-events/{self.date}'
        )
        response = conn.getresponse()
        data = response.read()
        data = json.loads(data)["events"]
        data = [{
            'tarih': self.date,
            'data': data
        }]
    
        if data is None:
            raise ValueError("Önce veriyi çekmek için 'fetch_data()' metodunu çalıştırın.")
        
        # Veriyi DataFrame'e çevir ve işle
        df = pd.DataFrame(data)
        df_exploded = df.explode('data')
        expanded_data = pd.json_normalize(df_exploded['data'])
        expanded_data['tarih'] = list(df_exploded['tarih'])
        datax = expanded_data.drop_duplicates(subset='id', keep='last')
        datax.columns = [item.replace(".", "_") for item in datax.columns]
        
        verim = datax.copy()

        return verim
        

    def fiksture_data(self):

            verim = self.fetch_data()
            """Eksik sütunları kontrol edip, varsa 0 ile doldurma ve veriyi sonlandırma."""
            if verim is None:
                raise ValueError("Veri işlenmedi. Önce 'process_data()' metodunu çalıştırın.")
            
            # Aynı uniqueTournament_id'ye sahip olanları tekrar kontrol et
            verim = verim.drop_duplicates("tournament_uniqueTournament_id")
            
            # İlgili sütunlar
            cols = [
                "tournament_uniqueTournament_id", "tournament_name", "tournament_slug", "tournament_category_name",
                "tournament_category_slug", "tournament_uniqueTournament_category_id", 
                "tournament_uniqueTournament_hasEventPlayerStatistics", "tournament_id", "tournament_isGroup", 
                "tournament_uniqueTournament_hasPerformanceGraphFeature"
            ]
            
            # Eksik sütunları 0 ile doldurma
            for col in cols:
                if col not in verim.columns:
                    verim.loc[:, col] = 0
    
            # Son olarak sadece belirtilen sütunlar kalsın
            verim = verim[cols]

            for _, row in verim.iterrows():
                tournament_id = row['tournament_uniqueTournament_id']
                
                # NaN değerlerini kontrol edin ve NaN olan satırları atlayın
                if pd.isna(tournament_id):
                    print(f"Geçersiz tournament_uniqueTournament_id: {tournament_id}, atlanıyor.")
                    continue  # NaN olan satırı atla
        
                # Modelde bu id ile kayıt olup olmadığını kontrol et
                if not Tournament.objects.filter(tournament_uniqueTournament_id=tournament_id).exists():
                    # Kayıt yoksa yeni kayıt oluştur
                    Tournament.objects.create(
                        tournament_uniqueTournament_id=row['tournament_uniqueTournament_id'],
                        tournament_name=row['tournament_name'],
                        tournament_slug=row['tournament_slug'],
                        tournament_category_name=row['tournament_category_name'],
                        tournament_category_slug=row['tournament_category_slug'],
                        tournament_uniqueTournament_category_id=row['tournament_uniqueTournament_category_id'],
                        tournament_uniqueTournament_hasEventPlayerStatistics=row['tournament_uniqueTournament_hasEventPlayerStatistics'],
                        tournament_id=row['tournament_id'],
                        tournament_isGroup=row['tournament_isGroup'] if pd.notna(row['tournament_isGroup']) else False,
                        tournament_uniqueTournament_hasPerformanceGraphFeature=row['tournament_uniqueTournament_hasPerformanceGraphFeature'] if pd.notna(row['tournament_uniqueTournament_hasPerformanceGraphFeature']) else False
                    )
                    print(f"Tournament ID {tournament_id} başarıyla eklendi.")
                else:
                    print(f"Tournament ID {tournament_id} zaten mevcut, eklenmedi.")

    def season_data(self):
        """Season verileri için eksik sütunları kontrol edip, 0 ile doldurma işlemi."""
        verim = self.fetch_data()
        
        if verim is None:
            raise ValueError("Veri işlenmedi. Önce 'process_data()' metodunu çalıştırın.")
        
        # Aynı season_id'ye sahip olanları tekrar kontrol et
        verim = verim.drop_duplicates("season_id")
        
        # Sezonla ilgili sütunlar
        season_cols = ["season_id", "season_name", "season_year","tournament_uniqueTournament_id"]

        # Eksik sütunları 0 ile doldurma
        for col in season_cols:
            if col not in verim.columns:
                verim.loc[:, col] = 0

        # Son olarak sadece season ile ilgili sütunları döndür
        verim = verim[season_cols]

        for _, row in verim.iterrows():
            season_id = row['season_id']
            tournament_id = row.get('tournament_uniqueTournament_id')  # İlgili turnuvayı al
    
            # NaN değerlerini kontrol edin ve geçersiz veriyi atlayın
            if pd.isna(season_id) or pd.isna(tournament_id):
                print(f"Geçersiz season_id veya tournament_id: Season ID={season_id}, Tournament ID={tournament_id}. Atlanıyor.")
                continue
    
            # Modelde bu id ile kayıt olup olmadığını kontrol et
            if not Season.objects.filter(season_id=season_id).exists():
                # Turnuva olup olmadığını kontrol et ve turnuva ile sezonu ilişkilendir
                tournament_instance = Tournament.objects.filter(tournament_uniqueTournament_id=tournament_id).first()
                
                if tournament_instance:
                    # Kayıt yoksa yeni sezon kaydı oluştur
                    Season.objects.create(
                        season_id=row['season_id'],
                        season_name=row['season_name'],
                        season_year=row['season_year'],
                        tournament=tournament_instance  # Turnuva ile ilişkilendir
                    )
                    print(f"Season ID {season_id} başarıyla eklendi.")
                else:
                    print(f"Tournament ID {tournament_id} bulunamadı, Season ID {season_id} eklenemedi.")
            else:
                print(f"Season ID {season_id} zaten mevcut, eklenmedi.")

    def team_data(self):
        """Verilen verilerden homeTeam bilgilerini çek ve modele ekle."""
        
        verim = self.fetch_data()  # Veriyi sofascore API'den çek

        if verim is None:
            raise ValueError("Veri işlenmedi. Önce 'fetch_data()' metodunu çalıştırın.")
        
        # Aynı homeTeam_id'ye sahip olanları tekrar kontrol et
        verim = verim.drop_duplicates("homeTeam_id")

        # İlgili home team sütunları
        homecols = [
            "homeTeam_id", "homeTeam_name", "homeTeam_slug", "homeTeam_shortName",
            "homeTeam_nameCode", "homeTeam_national", "tournament_id", "season_id"
        ]

        # Eksik sütunları kontrol edip, eğer yoksa 0 ile doldurma
        for col in homecols:
            if col not in verim.columns:
                verim.loc[:, col] = 0

        # Sadece home team ile ilgili sütunları al
        verim = verim[homecols]

        

        for _, row in verim.iterrows():
      
            
                tournament_instance = row['tournament_id']
                season_instance = row['season_id']
          
                Team.objects.create(
                    team_id=2820,
                    team_name="Sample Team",
                    team_slug="sample-team",
                    team_shortName="ST",
                    team_nameCode="ST",
                    team_national=False,
                    tournament_id=tournament_instance,
                    season_id=season_instance
                )
          

   
tarih = "2023-10-20"

api = SofascoreAPI(tarih)  
api.team_data()  # Veriyi sofascore API'den çeker        # Çekilen veriyi işler





IntegrityError: FOREIGN KEY constraint failed

In [42]:
api.season_data()       # Sezon verileriyle ilgili işlem


Season ID 52186 zaten mevcut, eklenmedi.
Season ID 52376 zaten mevcut, eklenmedi.
Season ID 52760 zaten mevcut, eklenmedi.
Season ID 52608 zaten mevcut, eklenmedi.
Season ID 52571 zaten mevcut, eklenmedi.
Season ID 48982 zaten mevcut, eklenmedi.
Season ID 53190 zaten mevcut, eklenmedi.
Season ID 52383 zaten mevcut, eklenmedi.
Season ID 53223 zaten mevcut, eklenmedi.
Season ID 47644 zaten mevcut, eklenmedi.
Season ID 52052 zaten mevcut, eklenmedi.
Season ID 52147 zaten mevcut, eklenmedi.
Season ID 53417 zaten mevcut, eklenmedi.
Season ID 52367 zaten mevcut, eklenmedi.
Season ID 52563 zaten mevcut, eklenmedi.
Season ID 52607 zaten mevcut, eklenmedi.
Season ID 52947 zaten mevcut, eklenmedi.
Season ID 52572 zaten mevcut, eklenmedi.
Season ID 49058 zaten mevcut, eklenmedi.
Season ID 52556 zaten mevcut, eklenmedi.
Season ID 53276 zaten mevcut, eklenmedi.
Season ID 54447 zaten mevcut, eklenmedi.
Season ID 49950 zaten mevcut, eklenmedi.
Season ID 52847 zaten mevcut, eklenmedi.
Season ID 52364 

In [45]:
api.team_data() 

Tournament ID 1 veya Season ID 52186 bulunamadı, Home Team ID 44 eklenemedi.
Tournament ID 36 veya Season ID 52376 bulunamadı, Home Team ID 2820 eklenemedi.
Tournament ID 36 veya Season ID 52376 bulunamadı, Home Team ID 2824 eklenemedi.
Tournament ID 33 veya Season ID 52760 bulunamadı, Home Team ID 2701 eklenemedi.
Home Team ID 2673 zaten mevcut, eklenmedi.
Home Team ID 2547 zaten mevcut, eklenmedi.
Home Team ID 2576 zaten mevcut, eklenmedi.
Home Team ID 2538 zaten mevcut, eklenmedi.
Home Team ID 2569 zaten mevcut, eklenmedi.
Home Team ID 2524 zaten mevcut, eklenmedi.
Tournament ID 4 veya Season ID 52571 bulunamadı, Home Team ID 1662 eklenemedi.
Tournament ID 83 veya Season ID 48982 bulunamadı, Home Team ID 1954 eklenemedi.
Tournament ID 83 veya Season ID 48982 bulunamadı, Home Team ID 1963 eklenemedi.
Tournament ID 83 veya Season ID 48982 bulunamadı, Home Team ID 1968 eklenemedi.
Tournament ID 83 veya Season ID 48982 bulunamadı, Home Team ID 1961 eklenemedi.
Tournament ID 62 veya Seas

In [12]:
from mymodel.models import Tournament, Season
from django.db.models import Prefetch

def get_tournament_and_season_data():
    """
    Tournament ve Season modellerinden verileri çekip birleştirir.
    """
    # Tournament ile ilişkili Season verilerini alıyoruz
    tournaments = Tournament.objects.prefetch_related(
        Prefetch('seasons', queryset=Season.objects.all())
    ).all()

    # Veriyi birleştirme ve liste oluşturma
    combined_data = []
    
    for tournament in tournaments:
        for season in tournament.seasons.all():  # İlişkili tüm sezonları alıyoruz
            combined_data.append({
                'tournament_id': tournament.tournament_uniqueTournament_id,
                'tournament_name': tournament.tournament_name,
                'season_id': season.season_id,
                'season_name': season.season_name,
                'season_year': season.season_year,
                'tournament_slug':tournament.tournament_slug,
                'tournament_category_name':tournament.tournament_category_name
            })
    
    return combined_data

# Kullanım
data = get_tournament_and_season_data()
dfx = pd.DataFrame(data)
dfx


,tournament_id,tournament_name,season_id,season_name,season_year,tournament_slug,tournament_category_name
0,7,UEFA Champions League,61644,UEFA Champions League 24/25,24/25,uefa-champions-league,Europe
1,7,UEFA Champions League,36886,UEFA Champions League 21/22,21/22,uefa-champions-league,Europe
2,7,UEFA Champions League,29267,UEFA Champions League 20/21,20/21,uefa-champions-league,Europe
3,17,Premier League,61627,Premier League 24/25,24/25,premier-league,England
4,17,Premier League,52186,Premier League 23/24,23/24,premier-league,England
...,...,...,...,...,...,...,...
1123,822,U19 1st Division,37557,U19 1st Division 21/22,21/22,u19-1st-division,Czech Republic
1124,535,"Danmarksserien, Pulje 3",37703,"Danmarksserien, Pulje 3 21/22",21/22,danmarksserien-pulje-3,Denmark Amateur
1125,11338,Azadegan League,39186,Azadegan League 21/22,21/22,azadegan-league,Iran
1126,195,FAI Cup,37545,FAI Cup 2021,2021,fai-cup,Ireland


In [6]:
dfs = dfx[dfx["tournament_category_name"]=="Turkey"]
dfs

,tournament_id,tournament_name,season_id,season_name,season_year,tournament_slug,tournament_category_name
31,52,Super Lig,63814,Super Lig 24/25,24/25,trendyol-super-lig,Turkey
32,52,Super Lig,53190,Super Lig 23/24,23/24,trendyol-super-lig,Turkey
33,52,Super Lig,42632,Super Lig 22/23,22/23,trendyol-super-lig,Turkey
34,52,Super Lig,17762,Super Lig 18/19,18/19,trendyol-super-lig,Turkey
35,52,Super Lig,29506,Super Lig 20/21,20/21,trendyol-super-lig,Turkey
36,52,Super Lig,37466,Super Lig 21/22,21/22,trendyol-super-lig,Turkey
37,52,Super Lig,10470,Super Lig 15/16,15/16,trendyol-super-lig,Turkey
86,98,1. Lig,64425,1. Lig 24/25,24/25,trendyol-1lig,Turkey
87,98,1. Lig,53276,1. Lig 23/24,23/24,trendyol-1lig,Turkey
88,98,1. Lig,42678,1. Lig 22/23,22/23,trendyol-1lig,Turkey


In [13]:
api = "https://www.sofascore.com/api/v1/unique-tournament/52/season/10470/events/round/4"

conn = http.client.HTTPSConnection('www.sofascore.com')
conn.request(
            'GET',api
            
        )
response = conn.getresponse()
data = response.read()
data = json.loads(data)["events"]
data = pd.DataFrame(data)
data

,tournament,season,roundInfo,customId,status,winnerCode,homeTeam,awayTeam,homeScore,awayScore,...,detailId,crowdsourcingDataDisplayEnabled,id,homeRedCards,startTimestamp,slug,finalResultOnly,feedLocked,isEditor,awayRedCards
0,"{'name': 'Super Lig', 'slug': 'trendyol-super-...","{'name': 'Super Lig 15/16', 'year': '15/16', '...",{'round': 4},klbsYUj,"{'code': 100, 'description': 'Ended', 'type': ...",3,"{'name': 'Akhisarspor', 'slug': 'akhisarspor',...","{'name': 'Gaziantepspor', 'slug': 'gaziantepsp...","{'current': 0, 'display': 0, 'period1': 0, 'pe...","{'current': 0, 'display': 0, 'period1': 0, 'pe...",...,1,False,6802029,1.0,1442066400,akhisarspor-gaziantepspor,False,False,False,NaN
1,"{'name': 'Super Lig', 'slug': 'trendyol-super-...","{'name': 'Super Lig 15/16', 'year': '15/16', '...",{'round': 4},flbscgd,"{'code': 100, 'description': 'Ended', 'type': ...",1,"{'name': 'Bursaspor', 'slug': 'bursaspor', 'sh...","{'name': 'Gençlerbirliği', 'slug': 'genclerbir...","{'current': 3, 'display': 3, 'period1': 1, 'pe...","{'current': 2, 'display': 2, 'period1': 0, 'pe...",...,1,False,6802026,NaN,1442066400,genclerbirligi-bursaspor,False,False,False,NaN
2,"{'name': 'Super Lig', 'slug': 'trendyol-super-...","{'name': 'Super Lig 15/16', 'year': '15/16', '...",{'round': 4},glbsoCc,"{'code': 100, 'description': 'Ended', 'type': ...",1,"{'name': 'Antalyaspor', 'slug': 'antalyaspor',...","{'name': 'Eskişehirspor', 'slug': 'eskisehirsp...","{'current': 2, 'display': 2, 'period1': 0, 'pe...","{'current': 0, 'display': 0, 'period1': 0, 'pe...",...,1,False,6802025,NaN,1442078100,eskisehirspor-antalyaspor,False,False,False,NaN
3,"{'name': 'Super Lig', 'slug': 'trendyol-super-...","{'name': 'Super Lig 15/16', 'year': '15/16', '...",{'round': 4},llbscmb,"{'code': 100, 'description': 'Ended', 'type': ...",3,"{'name': 'Galatasaray', 'slug': 'galatasaray',...","{'name': 'Mersin İdman Yurdu', 'slug': 'mersin...","{'current': 1, 'display': 1, 'period1': 0, 'pe...","{'current': 1, 'display': 1, 'period1': 0, 'pe...",...,1,False,6802024,1.0,1442078100,mersin-idman-yurdu-galatasaray,False,False,False,1.0
4,"{'name': 'Super Lig', 'slug': 'trendyol-super-...","{'name': 'Super Lig 15/16', 'year': '15/16', '...",{'round': 4},albsLlb,"{'code': 100, 'description': 'Ended', 'type': ...",1,"{'name': 'Beşiktaş', 'slug': 'besiktas', 'shor...","{'name': 'Başakşehir FK', 'slug': 'basaksehir-...","{'current': 2, 'display': 2, 'period1': 1, 'pe...","{'current': 0, 'display': 0, 'period1': 0, 'pe...",...,1,False,6802017,NaN,1442152800,basaksehir-fk-besiktas,False,False,False,NaN
5,"{'name': 'Super Lig', 'slug': 'trendyol-super-...","{'name': 'Super Lig 15/16', 'year': '15/16', '...",{'round': 4},olbsBlb,"{'code': 100, 'description': 'Ended', 'type': ...",3,"{'name': 'Çaykur Rizespor', 'slug': 'caykur-ri...","{'name': 'Sivasspor', 'slug': 'sivasspor', 'sh...","{'current': 1, 'display': 1, 'period1': 0, 'pe...","{'current': 1, 'display': 1, 'period1': 1, 'pe...",...,1,False,6802027,1.0,1442152800,sivasspor-caykur-rizespor,False,False,False,NaN
6,"{'name': 'Super Lig', 'slug': 'trendyol-super-...","{'name': 'Super Lig 15/16', 'year': '15/16', '...",{'round': 4},hlbsKlb,"{'code': 100, 'description': 'Ended', 'type': ...",2,"{'name': 'Sincan Belediyesi Ankaraspor', 'slug...","{'name': 'Konyaspor', 'slug': 'konyaspor', 'sh...","{'current': 1, 'display': 1, 'period1': 1, 'pe...","{'current': 2, 'display': 2, 'period1': 0, 'pe...",...,1,False,6802022,NaN,1442164500,konyaspor-sincan-belediyesi-ankaraspor,False,False,False,1.0
7,"{'name': 'Super Lig', 'slug': 'trendyol-super-...","{'name': 'Super Lig 15/16', 'year': '15/16', '...",{'round': 4},clbsnwc,"{'code': 100, 'description': 'Ended', 'type': ...",2,"{'name': 'Kasımpaşa', 'slug': 'kasimpasa', 'sh...","{'name': 'Fenerbahçe', 'slug': 'fenerbahce', '...","{'current': 0, 'display': 0, 'period1': 0, 'pe...","{'current': 1, 'display': 1, 'period1': 1, 'pe...",...,1,False,6802023,1.0,1442167200,kasimpasa-fenerbahce,False,False,F